<a href="https://colab.research.google.com/github/ImineAmazigh/MadLibs/blob/main/PricePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
#these are just libraries, that will be used, if you want to install more go to terminal or tell me
import tensorflow as tf
import optuna
import pandas as pd
import numpy as np

from google.colab import files

In [41]:
#our code starts here
df = pd.read_csv('/content/train_car.csv')
print(df)
id_train = df[['id']]
print(id_train.to_numpy(dtype= int32))
model_train = df[['model']]
print(model_train)
mileage_km_train = df[['mileage_km']]
print(mileage_km_train)
year_train = df[['year']]
print(year_train)
transmission_train = df[['transmission']]
print(transmission_train)
fuel_type_train = df[['fuel_type']]
print(fuel_type_train)
listing_date_train = df[['listing_date']]
print(listing_date_train)

matriceHolder = np.array[()]

print("we'll separate our data to 80% training and 20% testing")



         id                     model  mileage_km  year transmission  \
0      5895                CITROEN C3     45000.0  2021     Manuelle   
1      2926                    BMW X3    149000.0  2019  Automatique   
2      7027                   AUDI Q5    169000.0  2015  Automatique   
3      1765              DACIA Duster    235000.0  2021     Manuelle   
4      8931                OPEL Corsa     70000.0  2021     Manuelle   
...     ...                       ...         ...   ...          ...   
9577   5283  MERCEDES-BENZ Classe cla    120000.0  2017  Automatique   
9578  13643             DACIA Sandero       115.0  2018     Manuelle   
9579   5483                   AUDI Q3    180000.0  2012  Automatique   
9580    879                   AUDI Q3    109000.0  2022  Automatique   
9581   7390      MITSUBISHI Outlander    159000.0  2015  Automatique   

     fuel_type listing_date  tax_hp  doors   price  
0       Diesel   2025-02-14     6.0    5.0  142000  
1       Diesel   2025-10-29  

NameError: name 'int32' is not defined

In [39]:
def objective(trial) :
  lrBest = trial.suggest_float('trial', 1e-4, 1e-1)
  echosBest = trial.suggest_int('echosBest', 1, 30)
  n1Best = trial.suggest_int('n1Best')
  n2Best = trial.suggest_int('n2Best')
  model = tf.keras.Sequential([tf.keras.Input(shape= (10,)),
                               tf.keras.layers.Dense(n1Best, activation='relu'),
                               tf.keras.layers.Dense(n2Best,activation='relu'),
                               tf.keras.layers.Dense(1)])
  model.compile(optimizer='adamw',
                metrics=[tf.keras.metrics.RootMeanSquaredError()], )
  model.fit()
new_study = optuna.create_study()
new_study.optimize(objective, n_trials=2)



SyntaxError: invalid syntax (ipython-input-2153884575.py, line 4)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')